In [4]:
%pip install langchain-community 

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.graphs import Neo4jGraph
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Vietnam@"

graph = Neo4jGraph()

### Dataset: https://huggingface.co/datasets/tmnam20/ViMedAQA

In [6]:
import requests
import pandas as pd

# Define the URL and parameters
url = "https://datasets-server.huggingface.co/rows"
params = {
    "dataset": "tmnam20/ViMedAQA",
    "config": "disease",              # this matches the viewer page path
    "split": "train",
    "offset": 0,
    "length": 100
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a Pandas DataFrame
    rows = data.get('rows', [])
    df = pd.DataFrame([row['row'] for row in rows])

In [7]:
# df.head(5) 
df.columns # In thử 5 dòng đầu tiên của DataFrame

Index(['question_idx', 'question', 'answer', 'context', 'title', 'keyword',
       'topic', 'article_url', 'author', 'author_url'],
      dtype='object')

In [8]:
import pandas as pd
from langchain.docstore.document import Document


# Define the function to format each row
def format_row(row):
    return (
        f"Question: {row['question']}\n"
        f"Answer: {row['answer']}\n"
        f"Context: {row['context']}\n"
        f"Topic: {row['topic']}\n"
        f"Keywords: {row['keyword']}\n"
        f"Author: {row['author']}\n"
        f"Title: {row['title']}\n\n"
    ).lower()

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

#Convert the formatted text into a list of Document objects
documents = []
for text in df['formatted_text'].head(50).tolist():
    document = Document(page_content=text)
    documents.append(document)


In [9]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(documents)

print("Documents after splitting:")
for doc in documents:
    print(doc.page_content)

Documents after splitting:
question: khi soi bằng đèn thường, triệu chứng điển hình của viêm mống mắt là gì?
answer: đồng tử 2 bên không đều nhau, đồng tử bên bệnh nhỏ hơn bên bình thường.
context: 5.1 nhìn về mặt đại thể (khi soi bằng đèn thường) - đồng tử 2 bên không đều nhau (bên bệnh nhỏ hơn) có thể phát hiện đồng tử co bên bệnh lý co nhiều hơn bên bình thường, điều này có thể do viêm nhiều dẫn đến dính thành sau mống mắt lại với nhau và với thuỷ tinh thể. thông thường kích thường đồng tử là 2 – 3 mm, đều 2 bên. - mủ ở đáy mống mắt (mủ tiền phòng) đây là hiện tượng các tế bào viêm c�
�n tượng các tế bào viêm cùng với chất tiết (chứa nhiều fibrin). mủ nằm ở dưới là do nguyên nhân trọng lực. 5.2 khám mắt qua đèn khe: (split lamp) đây là công cụ chuyên dụng của bác sĩ chuyên khoa mắt. qua đèn khe, bác sĩ có thể phát hiện: - các tế bào nằm giữa giác mạc và mống mắt số lượng tế bào này trên diện tích 1 mm² giúp phân độ nặng của viêm màng bồ đào. - độ mờ thuỷ dịch như đã miêu tả, thuỷ dị

In [10]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi")
llm.invoke("Tell me a joke")

C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_68592\3729597464.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi")


' AI '

In [12]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.schema import Document

# Khởi tạo LLM transformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Lấy subset tài liệu (bạn có thể thay đổi số lượng nếu muốn)
subset_documents = documents[:30]

# Kiểm tra định dạng tài liệu đầu vào
for i, doc in enumerate(subset_documents):
    if not isinstance(doc, Document):
        raise ValueError(f"Tài liệu số {i} không phải là một instance của langchain.schema.Document")
    if not isinstance(doc.page_content, str):
        raise TypeError(f"page_content của tài liệu {i} không phải chuỗi. Loại: {type(doc.page_content)}")

# Chuyển từng document một để dễ kiểm soát lỗi
graph_documents = []
for i, doc in enumerate(subset_documents):
    try:
        gdoc = llm_transformer.convert_to_graph_documents([doc])[0]
        
        # Làm sạch relationships
        for rel in gdoc.relationships:
            if isinstance(rel["head"], dict):
                print(f"Document {i}: head là dict, chuyển thành chuỗi.")
                rel["head"] = str(rel["head"])
            if isinstance(rel["tail"], dict):
                print(f"Document {i}: tail là dict, chuyển thành chuỗi.")
                rel["tail"] = str(rel["tail"])
            if not rel["head"] or not rel["tail"]:
                print(f"Warning: Relationship rỗng trong document {i}.")
        
        graph_documents.append(gdoc)
    
    except Exception as e:
        print(f"Lỗi khi xử lý document {i}: {e}")

# Đẩy dữ liệu lên Neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=False,
    include_source=True
)


Lỗi khi xử lý document 2: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 3: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 4: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 6: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 8: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 11: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 13: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 18: unhashable type: 'dict'
Lỗi khi xử lý document 20: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 21: unhashable type: 'dict'
Lỗi khi xử lý document 23: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 26: 'Relationship' object is not subscriptable
Lỗi khi xử lý document 29: 'Relationship' object is not subscriptable


In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-embedding" # hoặc "cpu"
)

vector = embeddings.embed_query("Test embedding")
print(vector[:10])  # In thử 10 phần tử đầu


C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_68592\4138896556.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\COHOTECH\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\COHOTECH\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_n

[0.5637586712837219, -0.045386429876089096, 0.06224380061030388, -0.35753557085990906, -0.1838540881872177, 0.4406045973300934, 0.22847045958042145, -0.5542489290237427, -0.29756036400794983, 0.059878770262002945]


In [14]:
from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

In [16]:
df.head(20)  # In thử 5 dòng đầu tiên của DataFrame

,question_idx,question,answer,context,title,keyword,topic,article_url,author,author_url,formatted_text
0,disease_13345,"Khi soi bằng đèn thường, triệu chứng điển hình...","Đồng tử 2 bên không đều nhau, đồng tử bên bệnh...",5.1 Nhìn về mặt đại thể (khi soi bằng đèn thườ...,Triệu chứng - Triệu chứng qua thăm khám,Viêm mống mắt,1,https://youmed.vn/tin-tuc/viem-mong-mat-nguyen...,Bác sĩ Nguyễn Đoàn Trọng Nhân,https://youmed.vn/tin-tuc/bac-si/nguyen-doan-t...,"question: khi soi bằng đèn thường, triệu chứng..."
1,disease_14890,Áp xe hậu môn là gì?,Áp xe hậu môn là một ổ nhiễm trùng có chứa mủ ...,Áp xe là một ổ nhiễm trùng có chứa mủ gần hậu ...,Áp xe và lỗ rò hậu môn,Đau hậu môn,1,https://youmed.vn/tin-tuc/dau-hau-mon-chuyen-k...,Bác sĩ Nguyễn Hồ Thanh An,https://youmed.vn/tin-tuc/bac-si/bac-si-nguyen...,question: áp xe hậu môn là gì?\nanswer: áp xe ...
2,disease_2538,Tỷ lệ trẻ được chẩn đoán bệnh Hirschsprung tro...,Khoảng 80% trẻ được chẩn đoán bệnh trong năm đ...,Các triệu chứng có thể khác nhau từ trẻ này sa...,Triệu chứng bệnh,Hirschsprung,1,https://youmed.vn/tin-tuc/benh-hirschsprung/,ThS.BS Nguyễn Hồng Vân Khánh,https://youmed.vn/tin-tuc/bac-si/thac-si-bac-s...,question: tỷ lệ trẻ được chẩn đoán bệnh hirsch...
3,disease_1455,"Khi bị cơn mất trí nhớ thoáng qua, người bệnh ...","Trong cơn thiếu máu não thoáng qua, người bệnh...",Cơn mất trí nhớ thoáng qua là một tình trạng m...,Cơn mất trí nhớ thoáng qua là gì?,Cơn mất trí nhớ thoáng qua,1,https://youmed.vn/tin-tuc/con-mat-tri-nho-thoa...,Bác sĩ Nguyễn Văn Huấn,https://youmed.vn/tin-tuc/bac-si/bac-si-nguyen...,"question: khi bị cơn mất trí nhớ thoáng qua, n..."
4,disease_15268,Thuốc nào được sử dụng để điều trị đái tháo nh...,Thuốc hydrochlorothiazide.,"Nếu bệnh đái tháo nhạt ở mức độ nhẹ, bạn chỉ c...",Điều trị đái tháo nhạt như thế nào? - Đái tháo...,Đái tháo nhạt,1,https://youmed.vn/tin-tuc/nhung-dieu-ban-can-b...,ThS.BS Vũ Thành Đô,https://youmed.vn/tin-tuc/bac-si/bac-si-vu-tha...,question: thuốc nào được sử dụng để điều trị đ...
5,disease_2317,Đâu là phương pháp chẩn đoán hiệu quả trong tr...,Chụp MRI hoặc CT scan sọ não.,Chụp MRI hoặc CT scan sọ não để loại trừ các ...,Chẩn đoán Giả u não bằng cách nào? - Hình ảnh ...,Giả u não,1,https://youmed.vn/tin-tuc/benh-gia-u-nao/,Bác sĩ Nguyễn Văn Huấn,https://youmed.vn/tin-tuc/bac-si/bac-si-nguyen...,question: đâu là phương pháp chẩn đoán hiệu qu...
6,disease_6321,Những trường hợp nào không nên tiêm vắc xin ph...,Phụ nữ đang mang thai và không có thai ít nhất...,Bệnh Rubella ở trẻ sau khi sinh và người trưởn...,Phòng ngừa bệnh Rubella?,Rubella,1,https://youmed.vn/tin-tuc/rubella-dieu-tri-va-...,Bác sĩ Võ Thị Ngọc Hiền,https://youmed.vn/tin-tuc/bac-si/bac-si-vo-thi...,question: những trường hợp nào không nên tiêm ...
7,disease_3514,Trong trường hợp nào bệnh nhân có thể kiểm soá...,Nếu bệnh nhân có cục máu đông không ảnh hưởng ...,Mức độ của bệnh sẽ thay đổi tuỳ thuộc vào cơ q...,Hội chứng kháng phospholipid có nguy hiểm không?,Hội chứng kháng Phospholipid,1,https://youmed.vn/tin-tuc/hoi-chung-khang-phos...,Bác sĩ Đinh Gia Khánh,https://youmed.vn/tin-tuc/bac-si/bac-si-dinh-g...,question: trong trường hợp nào bệnh nhân có th...
8,disease_2415,Gãy xương cổ chân thường là tổn thương những x...,Gãy xương cổ chân thường là tổn thương một hay...,Gãy xương cổ chân là tổn thương một hay nhiều ...,Thông tin chung về gãy xương cổ chân,Gãy xương cổ chân,1,https://youmed.vn/tin-tuc/gay-xuong-co-chan/,Bác sĩ Hồ Đức Việt,https://youmed.vn/tin-tuc/bac-si/bac-si-ho-duc...,question: gãy xương cổ chân thường là tổn thươ...
9,disease_11461,Liệu pháp nhắm trúng đích trong điều trị ung t...,Liệu pháp nhắm trúng đích là liệu pháp điều tr...,Hoá trị sử dụng các thuốc chống ung thư để tiê...,Điều trị ung thư não thứ phát - Hoá trị liệu,Ung thư não thứ phát,1,https://youmed.vn/tin-tuc/ung-thu-nao-thu-phat/,Bác sĩ Phan Văn Giáo,https://youmed.vn/tin-tuc/bac-si/bac-si-phan-v...,question: liệu pháp nhắm trúng đích trong điều...


In [17]:
df.question.unique()  # In thử các từ khóa duy nhất trong DataFrame

array(['Khi soi bằng đèn thường, triệu chứng điển hình của viêm mống mắt là gì?',
       'Áp xe hậu môn là gì?',
       'Tỷ lệ trẻ được chẩn đoán bệnh Hirschsprung trong năm đầu tiên của cuộc đời là bao nhiêu?',
       'Khi bị cơn mất trí nhớ thoáng qua, người bệnh sẽ gặp phải những triệu chứng nào?',
       'Thuốc nào được sử dụng để điều trị đái tháo nhạt do thận?',
       'Đâu là phương pháp chẩn đoán hiệu quả trong trường hợp bác sĩ nghi ngờ bệnh Giả u não?',
       'Những trường hợp nào không nên tiêm vắc xin phòng ngừa Rubella?',
       'Trong trường hợp nào bệnh nhân có thể kiểm soát hội chứng kháng phospholipid bằng thuốc uống và ngăn ngừa các biến chứng?',
       'Gãy xương cổ chân thường là tổn thương những xương nào?',
       'Liệu pháp nhắm trúng đích trong điều trị ung thư não là gì?',
       'Tại sao nam giới có nguy cơ mắc bệnh gout cao hơn phụ nữ?',
       'Khi nghi ngờ bị loét dạ dày tá tràng nên đến khoa nào tại bệnh viện để thăm khám?',
       'Bạn có thể giải thích 

In [18]:
import gradio as gr

# Define the function for querying patient details
def query_patient_details(query):
    try:
        result = qa_chain({"query": query})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=query_patient_details,        # Function to call
    inputs=gr.Textbox(label="Enter your question"),  # Input textbox
    outputs=gr.Textbox(label="Answer")   # Output textbox
)

# Launch the interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://caf8165e9f03770405.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\COHOTECH\AppData\Local\Temp\ipykernel_68592\3595798144.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Created dataset file at: .gradio\flagged\dataset1.csv
